**Немецко-ивритский переводчик, основанный на правилах**

Общий алгоритм выглядит следующим образом:
1. В качестве инпута переводчику скармливается текст на немецком 
2. Модель осуществляет препроцессинг:

    2.1. Сентенизацию (если текст состоит более, чем из одного предложения)

    2.2. Очистку от знаков препинания

    2.3. Токенизацию

    2.4. Лемматизацию

    2.5. POS-tagging и другие компоненты морфологической разметки

    2.6. Синтаксический парсинг: извлечение информации о синтаксических зависимостях и порядке слов.
    
3. Из словаря выбираются лексические соответствия для языка-реципиента с учётом морфологических признаков (если, например, на вход подается предложение с перфектной формой вида haben/sein + PII, осуществляется выбор ивритской формы не только с учётом TAME-категорий, но и с учётом морфологического кодирования рода S-участника: Er hat gesehen -> הוא ראה [hu ra'ah], Sie hat gesehen -> היא ראתה [hi raatah])
4. Переводчик применяет синтаксические правила языка-реципиента, формируя результат при помощи полученной информации о синтаксических зависимостях между лексическими единицами
5. В качестве аутпута пользователь получает текст на иврите, в котором соблюдён, в частности, корректный порядок слов, правила которого так же обозначаются в структуре кода


**Библиотеки для препроцессинга текстов**

В первую очередь, следует воспользоваться библиотекой stanza - для лемматизации и морфологической разметки. Выбор ее вместо привычных NLTK и SpaCy объясняется тем, что она предлагает модель для иврита. Библиотека предоставляет те же возможности, что и пакет UDPipe в R.

Для начала нужно загрузить библиотеки и создать пайплайны для обоих языков.

In [1]:
!pip install stanza

   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 1.1/1.1 MB 17.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/586.9 kB ? eta -:--:--
   --------------------------------------- 586.9/586.9 kB 21.7 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import stanza 

stanza.download('he')
stanza.download('de')

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-04 02:36:35 INFO: Downloaded file to C:\Users\User\stanza_resources\resources.json
2024-12-04 02:36:35 INFO: Downloading default packages for language: he (Hebrew) ...
2024-12-04 02:36:48 INFO: Downloaded file to C:\Users\User\stanza_resources\he\default.zip
2024-12-04 02:36:50 INFO: Finished downloading models and saved to C:\Users\User\stanza_resources
2024-12-04 02:36:50 INFO: Downloaded file to C:\Users\User\stanza_resources\resources.json
2024-12-04 02:36:50 INFO: Downloading default packages for language: de (German) ...
2024-12-04 02:37:16 INFO: Downloaded file to C:\Users\User\stanza_resources\de\default.zip
2024-12-04 02:37:21 INFO: Finished downloading models and saved to C:\Users\User\stanza_res

In [4]:
de = stanza.Pipeline('de')
he = stanza.Pipeline('he')

2024-12-04 02:41:17 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-12-04 02:41:17 INFO: Downloaded file to C:\Users\User\stanza_resources\resources.json
2024-12-04 02:41:18 INFO: Loading these models for language: de (German):
| Processor    | Package      |
-------------------------------
| tokenize     | gsd          |
| mwt          | gsd          |
| pos          | gsd_charlm   |
| lemma        | gsd_nocharlm |
| constituency | spmrl_charlm |
| depparse     | gsd_charlm   |
| sentiment    | sb10k_charlm |
| ner          | germeval2014 |

2024-12-04 02:41:18 INFO: Using device: cpu
2024-12-04 02:41:18 INFO: Loading: tokenize
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\stanza\models\tokenization\trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), w

Воспользовавшись кодом ниже, ознакомимся с характеристиками лексем, которые есть в тексте инпута

In [10]:
text = 'Guten Tag! Wie heißt du? Ich freue mich, dich kennenzulernen.'
doc = de(text)

for sent in doc.sentences:
    for word in sent.words:
        print(word)

{
  "id": 1,
  "text": "Guten",
  "lemma": "gut",
  "upos": "ADJ",
  "xpos": "ADJA",
  "feats": "Case=Acc|Degree=Pos|Gender=Masc|Number=Sing",
  "head": 2,
  "deprel": "amod",
  "start_char": 0,
  "end_char": 5
}
{
  "id": 2,
  "text": "Tag",
  "lemma": "Tag",
  "upos": "NOUN",
  "xpos": "NN",
  "feats": "Case=Acc|Gender=Masc|Number=Sing",
  "head": 0,
  "deprel": "root",
  "start_char": 6,
  "end_char": 9
}
{
  "id": 3,
  "text": "!",
  "lemma": "!",
  "upos": "PUNCT",
  "xpos": "$.",
  "head": 2,
  "deprel": "punct",
  "start_char": 9,
  "end_char": 10
}
{
  "id": 1,
  "text": "Wie",
  "lemma": "wie",
  "upos": "ADV",
  "xpos": "PWAV",
  "feats": "PronType=Int",
  "head": 2,
  "deprel": "advmod",
  "start_char": 11,
  "end_char": 14
}
{
  "id": 2,
  "text": "heißt",
  "lemma": "heißen",
  "upos": "VERB",
  "xpos": "VVFIN",
  "feats": "Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin",
  "head": 0,
  "deprel": "root",
  "start_char": 15,
  "end_char": 20
}
{
  "id": 3,
  "text": 

Выведем нужные характеристики 

In [11]:
for sent in doc.sentences:
    for word in sent.words:
        print(f'{word.text.lower()} : {word.lemma}, POS: {word.upos}, Gram: {word.feats}, Syntax: {word.deprel}')

guten : gut, POS: ADJ, Gram: Case=Acc|Degree=Pos|Gender=Masc|Number=Sing, Syntax: amod
tag : Tag, POS: NOUN, Gram: Case=Acc|Gender=Masc|Number=Sing, Syntax: root
! : !, POS: PUNCT, Gram: None, Syntax: punct
wie : wie, POS: ADV, Gram: PronType=Int, Syntax: advmod
heißt : heißen, POS: VERB, Gram: Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin, Syntax: root
du : du, POS: PRON, Gram: Case=Nom|Number=Sing|Person=2|PronType=Prs, Syntax: nsubj
? : ?, POS: PUNCT, Gram: None, Syntax: punct
ich : ich, POS: PRON, Gram: Case=Nom|Number=Sing|Person=1|PronType=Prs, Syntax: nsubj
freue : freuen, POS: VERB, Gram: Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin, Syntax: root
mich : ich, POS: PRON, Gram: Case=Acc|Number=Sing|Person=1|PronType=Prs|Reflex=Yes, Syntax: obj
, : ,, POS: PUNCT, Gram: None, Syntax: punct
dich : du, POS: PRON, Gram: Case=Acc|Number=Sing|Person=1|PronType=Prs, Syntax: obj
kennenzulernen : kennenlernen, POS: VERB, Gram: VerbForm=Inf, Syntax: xcomp
. : ., POS: PUNCT, 

Сделаем то же самое с текстом на иврите

In [14]:
text_1 = 'יום טוב! איך קוראים לך? אני שמח לפגוש אותך'
doc_1 = he(text_1)

for sent in doc_1.sentences:
    for word in sent.words:
        print(f'{word.text.lower()} : {word.lemma}, POS: {word.upos}, Gram: {word.feats}, Syntax: {word.deprel}')

יום : יום, POS: NOUN, Gram: Gender=Masc|Number=Sing, Syntax: obl:tmod
טוב : טוב, POS: ADJ, Gram: Gender=Masc|Number=Sing, Syntax: amod
! : !, POS: PUNCT, Gram: None, Syntax: punct
איך : איך, POS: ADV, Gram: PronType=Int, Syntax: advmod
קוראים : קרא, POS: VERB, Gram: Gender=Masc|HebBinyan=PAAL|Number=Plur|Person=3|Tense=Pres|VerbForm=Part|Voice=Act, Syntax: root
ל : ל, POS: ADP, Gram: None, Syntax: case
ך : הוא, POS: PRON, Gram: Gender=Masc|Number=Sing|Person=2|PronType=Prs, Syntax: obl
? : ?, POS: PUNCT, Gram: None, Syntax: punct
אני : הוא, POS: PRON, Gram: Gender=Masc|Number=Sing|Person=1|PronType=Prs, Syntax: nsubj
שמח : שמח, POS: VERB, Gram: Gender=Masc|HebBinyan=PAAL|Number=Sing|Person=1|Tense=Pres|VerbForm=Part|Voice=Act, Syntax: root
לפגוש : פגש, POS: VERB, Gram: HebBinyan=HIFIL|VerbForm=Inf|Voice=Act, Syntax: xcomp
אות : את, POS: ADP, Gram: Case=Acc, Syntax: case
ך : הוא, POS: PRON, Gram: Gender=Masc|Number=Sing|Person=2|PronType=Prs, Syntax: obj


Теперь мы имеем набор лемм и грамматических признаков для лексем в составе двух эквивалентных предложений.
Из них необходимо сформировать словарь, в котором будут представлены не только пословные соответствия, но и соответствия морфологических признаков и синтаксических ролей. Структура будет представлена в виде словаря словарей, где информация о грамматических признаках будет содержаться во внутреннем словаре. 

In [ ]:
word_info = {
    "guten": {
        "lemma": "gut",
        "POS": "ADJ",
        "Gram": "Case=Acc|Degree=Pos|Gender=Masc|Number=Sing",
        "Syntax": "amod",
        "hebrew": {
            "word": "טוב",
            "lemma": "טוב",
            "POS": "ADJ",
            "Gram": "Gender=Masc|Number=Sing",
            "Syntax": "amod"
        }
    },
    "tag": {
        "lemma": "Tag",
        "POS": "NOUN",
        "Gram": "Case=Acc|Gender=Masc|Number=Sing",
        "Syntax": "root",
        "hebrew": {
            "word": "יום",
            "lemma": "יום",
            "POS": "NOUN",
            "Gram": "Gender=Masc|Number=Sing",
            "Syntax": "root"
        }
    },
    "!": {
        "lemma": "!",
        "POS": "PUNCT",
        "Gram": "None",
        "Syntax": "punct",
        "hebrew": {
            "word": "!",
            "lemma": "!",
            "POS": "PUNCT",
            "Gram": "None",
            "Syntax": "punct"
        }
    },
    "wie": {
        "lemma": "wie",
        "POS": "ADV",
        "Gram": "PronType=Int",
        "Syntax": "advmod",
        "hebrew": {
            "word": "איך",
            "lemma": "איך",
            "POS": "ADV",
            "Gram": "PronType=Int",
            "Syntax": "advmod"
        }
    },
    "heißt": {
        "lemma": "heißen",
        "POS": "VERB",
        "Gram": "Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin",
        "Syntax": "root",
        "hebrew": {
            "word": "קוראים",
            "lemma": "קרא",
            "POS": "VERB",
            "Gram": "Gender=Masc|HebBinyan=PAAL|Number=Plur|Person=3|Tense=Pres|VerbForm=Part|Voice=Act",
            "Syntax": "root"
        }
    },
    "du": {
        "lemma": "du",
        "POS": "PRON",
        "Gram": "Case=Nom|Number=Sing|Person=2|PronType=Prs",
        "Syntax": "nsubj",
        "hebrew": {
            "word": "ך",
            "lemma": "הוא",
            "POS": "PRON",
            "Gram": "Gender=Masc|Number=Sing|Person=2|PronType=Prs",
            "Syntax": "obl"
        }
    },
    "?": {
        "lemma": "?",
        "POS": "PUNCT",
        "Gram": "None",
        "Syntax": "punct",
        "hebrew": {
            "word": "?",
            "lemma": "?",
            "POS": "PUNCT",
            "Gram": "None",
            "Syntax": "punct"
        }
    },
    "ich": {
        "lemma": "ich",
        "POS": "PRON",
        "Gram": "Case=Nom|Number=Sing|Person=1|PronType=Prs",
        "Syntax": "nsubj",
        "hebrew": {
            "word": "אני",
            "lemma": "אני",
            "POS": "PRON",
            "Gram": "Gender=Masc|Number=Sing|Person=1|PronType=Prs",
            "Syntax": "nsubj"
        }
    },
    "freue": {
        "lemma": "freuen",
        "POS": "VERB",
        "Gram": "Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin",
        "Syntax": "root",
        "hebrew": {
            "word": "שמח",
            "lemma": "שמח",
            "POS": "VERB",
            "Gram": "Gender=Masc|HebBinyan=PAAL|Number=Sing|Person=1|Tense=Pres|VerbForm=Part|Voice=Act",
            "Syntax": "root"
        }
    },
    "mich": {
        "lemma": "ich",
        "POS ": "PRON",
        "Gram": "Case=Acc|Number=Sing|Person=1|PronType=Prs|Reflex=Yes",
        "Syntax": "obj",
        "hebrew": {
            "word": "אותך",
            "lemma": "אותך",
            "POS": "PRON",
            "Gram": "Case=Acc|Number=Sing|Person=1|PronType=Prs|Reflex=Yes",
            "Syntax": "obj"
        }
    },
    ",": {
        "lemma": ",",
        "POS": "PUNCT",
        "Gram": "None",
        "Syntax": "punct",
        "hebrew": {
            "word": ",",
            "lemma": ",",
            "POS": "PUNCT",
            "Gram": "None",
            "Syntax": "punct"
        }
    },
    "dich": {
        "lemma": "du",
        "POS": "PRON",
        "Gram": "Case=Acc|Number=Sing|Person=1|PronType=Prs",
        "Syntax": "obj",
        "hebrew": {
            "word": "לך",
            "lemma": "לך",
            "POS": "PRON",
            "Gram": "Case=Acc|Number=Sing|Person=2|PronType=Prs",
            "Syntax": "obj"
        }
    },
    "kennenzulernen": {
        "lemma": "kennenlernen",
        "POS": "VERB",
        "Gram": "VerbForm=Inf",
        "Syntax": "xcomp",
        "hebrew": {
            "word": "לפגוש",
            "lemma": "לפגוש",
            "POS": "VERB",
            "Gram": "HebBinyan=HIFIL|VerbForm=Inf|Voice=Act",
            "Syntax": "xcomp"
        }
    },
    ".": {
        "lemma": ".",
        "POS": "PUNCT",
        "Gram": "None",
        "Syntax": "punct",
        "hebrew": {
            "word": ".",
            "lemma": ".",
            "POS": "PUNCT",
            "Gram": "None",
            "Syntax": "punct"
        }
    }
}

Поскольку морфология и синтаксис в иврите и немецком различаются, формирование словаря должно проводиться с опорой на следующие правила:

1. Проверяем часть речи: если одинаковая в обеих языках, сравниваем грамматические признаки; если разная, ищем другую лексему с совпадающей частеречной категорией. Добавляем в список возможных кандидатов все лексемы в тексте с их леммами и морфосинтаксическими характеристиками.

2. Проверка частей речи завершена - сравниваем грамматические признаки (например, для глагола): убираем те ивритские лексемы, лицо и число которых не совпадает с лицом и числом немецкой словоформы. Оставляем только совпадающие. В словарь ивритского аналога добавляем все возможные значения категории Gender. Так, например, для глагола heißen во вложенном словаре со значениями для иврита должно содержаться две лексемы - для мужского и женского родов. Соответственно, в грамматических категориях это тоже должно быть отражено (в идеале, должны быть учтены все омонимичные формы, поскольку глагол heißen в настоящем времени имеет одинаковую форму для второго и третьего лица единственного числа, а в иврите в настоящем времени формы глагола не дифференцируются по лицу: только по числу и роду). Таким образом, словарь для формы 'heißt' будет выглядеть следующим образом:


In [ ]:
word = {"heißt": {
        "lemma": "heißen",
        "POS": "VERB",
        "Gram": ["Mood=Ind|Number=Sing|Person=2|Tense=Pres|VerbForm=Fin", 
                 "Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin"],
        "Syntax": "root",
        "hebrew": {
            "word": ["קוֹרֵא", "קוֹרֵאת"],
            "lemma": "קרא", 
            "POS": "VERB",
            "Gram": ["Gender=Masc|HebBinyan=PAAL|Number=Sing|Tense=Pres|VerbForm=Part|Voice=Act", 
                     "Gender=Fem|HebBinyan=PAAL|Number=Plur|Tense=Pres|VerbForm=Part|Voice=Act"
                     ],
            "Syntax": "root"
        }
    }
}

То же самое касается местоимений второго лица: во вложенном словаре местоимениq du и ihr для иврита должны содержаться эквиваленты в мужском и женском роде, которые в тексте перевода должны быть выведены через слэш с соответствующиими пометками (m.) и (f.)

3. Если на расстоянии (3, 3) есть слово, грамматический род которого известен, и синтаксическая роль указывает на то, что оно связано с местоимением второго лица, из словаря при переводе должен быть выбран эквивалент, род которого соответствует роду маркированного слова, по которому опознается род местоимения.

4. После формирования списка лексем для вывода им должны быть присвоены индексы. Поскольку слова в иврите читаются справа налево, общее правило вывода текста перевода заключается в зеркальном позиционном кодировании: токен с индексом 0 в немецком соответствует токену с индексом -1 в иврите, первый токен в немецком тексте = минус второй токен в тексте на иврите, и т. д.

5. При переводе местоименных посессивных конструкций с генитивным аргументом (такие конструкции отслеживаем по значению ключа Syntax в словарях) вершина и зависимое должны меняться местами: в немецком языке порядок зависимое-вершина (meine Mutter), а в иврите - вершина-зависимое (/има (Mutter) шели (meine)/). 

На самом деле, правил для этой языковой пары должно быть очень много, но если перечислять все, то можно состариться... Хорошо, что придумали статистический и нейросетевой перевод.
